In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataval.dataset import WeatherDataset
from dataval.train import Trainer

import os
import matplotlib.pyplot as plt
import modal
import pandas as pd

In [12]:
image = (
    modal.Image.debian_slim()
    .pip_install_from_requirements("requirements.txt")
    .pip_install(["tensorflow-data-validation"])
)
stub = modal.Stub("tfdv-tutorial", image=image)

# Schema Validation

In this notebook, we will implement TFX's schema validation to drop rows that might be corrupted. We will use the same corruptions from the previous notebook, and measure how the validation improves MSE.

In [7]:
# Load dataset

ds = WeatherDataset(os.path.join(os.getcwd(), "canonical-partitioned-dataset"), sample_frac=0.2)

In [6]:
def train_and_test(train_df, test_df):
    X_train, y_train = ds.split_feature_label(train_df)

    catboost_hparams = {"depth": 5, "iterations": 250, "learning_rate": 0.03, "loss_function": "RMSE"}
    continual_t = Trainer(catboost_hparams)
    continual_t.fit(X_train, y_train, verbose=False)
    print(f"Train accuracy for partition {ds.get_partition_key(train_df)}: {continual_t.score(X_train, y_train)}")

    # Evaluate
    X_test, y_test = ds.split_feature_label(test_df)
    print(f"Test accuracy for partition {ds.get_partition_key(test_df)}: {continual_t.score(X_test, y_test)}")
    
    return continual_t

In [8]:
train_df = ds.load(ds.get_partition_keys()[0])
test_df = ds.load(ds.get_partition_keys()[1])

## Infer schema and check test data for errors

From the train dataframe, we create a schema using TFDV. Then we use this schema to find anomalies in the test data. We apply this to the original dataframes first.

In [23]:
@stub.function
def gen_schema_from_dataframe(dataframe):
    import tensorflow_data_validation as tfdv
    stats =  tfdv.generate_statistics_from_dataframe(dataframe)    
    schema = tfdv.infer_schema(statistics=stats)
    print(tfdv.display_schema(schema=schema))

In [24]:
with stub.run():
    gen_schema_from_dataframe.call(train_df)

✓ Initialized. View app at https://modal.com/apps/ap-ARTNE5TxIhZsQTSHIhfCeM

✓ Created objects.
├── 🔨 Created gen_statistics_from_dataframe.
└── 🔨 Created gen_schema_from_dataframe.

2023-03-18 00:56:28.485882: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-03-18 00:56:28.880455: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


2023-03-18 00:56:28.880527: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


...

2023-03-18 00:56:30.678109: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


                                     Type  Presence Valency       Domain
Feature name                                                            
'fact_latitude'                     FLOAT  required                    -
'fact_longitude'                    FLOAT  required                    -
'fact_temperature'                  FLOAT  required                    -
'fact_cwsm_class'                   FLOAT  required                    -
'climate'                          STRING  required            'climate'
...                                   ...       ...     ...          ...
'gfs_2m_dewpoint_next'              FLOAT  required                    -
'gfs_total_clouds_cover_low_grad'   FLOAT  required                    -
'gfs_total_clouds_cover_low_next'   FLOAT  required                    -
'year_week'                        STRING  required          'year_week'
'__index_level_0__'                   INT  required                    -

[130 rows x 4 columns]


                                          Values
Domain                                          
'climate'    'dry', 'mild temperate', 'tropical'
'year_week'                            '2018_35'


None


✓ App completed.

In [27]:
@stub.function
def find_anomalies(train_df, test_df):
    import tensorflow_data_validation as tfdv
    train_stats =  tfdv.generate_statistics_from_dataframe(train_df)
    schema = tfdv.infer_schema(statistics=train_stats)
    test_stats = tfdv.generate_statistics_from_dataframe(test_df)
    # tfdv.visualize_statistics(lhs_statistics=test_stats, rhs_statistics=train_stats, lhs_name='TEST_DATASET', rhs_name='TRAIN_DATASET')
    anomalies = tfdv.validate_statistics(statistics=test_stats, schema=schema)
    print(anomalies)

In [28]:
with stub.run():
    find_anomalies.call(train_df, test_df)

✓ Initialized. View app at https://modal.com/apps/ap-sOy0h2ted71biDtWrKNcRI

✓ Created objects.
├── 🔨 Created gen_statistics_from_dataframe.
├── 🔨 Created gen_schema_from_dataframe.
└── 🔨 Created find_anomalies.

2023-03-18 01:02:16.345628: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-03-18 01:02:16.689767: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


2023-03-18 01:02:16.844404: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


...

      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "gfs_temperature_sea"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "gfs_temperature_sea_grad"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "gfs_temperature_sea_interpolated"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "gfs_temperature_sea_next"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "gfs_timedelta_s"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "gfs_total_clouds_cover_high"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "gfs_total_clouds_cover_low"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "gfs_total_clouds_cover_middle"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "gfs_u_wind"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "gfs_v_wind"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "gfs_wind_speed"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }


    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "wrf_available"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "wrf_t2"
    value_count {
      min: 1
      max: 1
    }
    type: FLOAT
    presence {
      min_count: 1
    }
  }
  feature {
    name: "wrf_t2_next"
    value_count {
      min: 1
      max: 1
    }
    type: FLOAT
    presence {
      min_count: 1
    }
  }
  feature {
    name: "wrf_psfc"
    value_count {
      min: 1
      max: 1
    }
    type: FLOAT
    presence {
      min_count: 1
    }
  }
  feature {
    name: "wrf_rh2"
    value_count {
      min: 1
      max: 1
    }
    type: FLOAT
    presence {
      min_count: 1
    }
  }
  feature {
    name: "wrf_wind_u"
    value_count {
      min: 1
      max: 1
    }
    type: FLOAT
    presence {
      min_count: 1
    }
  }
  feature {
    name: "wrf_wind_v"
    value_count {
      min: 1
      max: 1
    }
    type: FLOAT
    presence {
      min_count: 1
    }
  }
  feature {
    name: "wrf_rain"
    value_count {
      min: 1
      max: 1
    }
    type: FLOAT
    presence {
      min_count: 1
    }
  }
  feature {
    name: "wrf_snow"
    value_count {
      min: 1
      max: 1
    }
    type: FLOAT
    presence {
      min_count: 1
    }
  }
  feature {
    name: "wrf_graupel"
    value_count {
      min: 1
      max: 1
    }
    type: FLOAT
    presence {
      min_count: 1
    }
  }
  feature {
    name: "wrf_hail"
    value_count {
      min: 1
      max: 1
    }
    type: FLOAT
    presence {
      min_count: 1
    }
  }
  feature {
    name: "wrf_t2_interpolated"
    value_count {
      min: 1
      max: 1
    }
    type: FLOAT
    presence {
      min_count: 1
    }
  }
  feature {
    name: "wrf_t2_grad"
    value_count {
      min: 1
      max: 1
    }
    type: FLOAT
    presence {
      min_count: 1
    }
  }
  feature {
    name: "cmc_0_1_65_0_grad"
    type: FLOAT
    presence {


      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "cmc_0_1_65_0_next"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "cmc_0_1_66_0_grad"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "cmc_0_1_66_0_next"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "cmc_0_1_67_0_grad"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "cmc_0_1_67_0_next"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "cmc_0_1_68_0_grad"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "cmc_0_1_68_0_next"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "gfs_2m_dewpoint_grad"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "gfs_2m_dewpoint_next"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "gfs_total_clouds_cover_low_grad"
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "gfs_total_clouds_cover_low_next"
    type: FLOAT
    presence {
      min_fraction: 1.0


      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "year_week"
    type: BYTES
    domain: "year_week"
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  feature {
    name: "__index_level_0__"
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
    shape {
      dim {
        size: 1
      }
    }
  }
  string_domain {
    name: "climate"
    value: "dry"
    value: "mild temperate"
    value: "tropical"
  }
  string_domain {
    name: "year_week"
    value: "2018_35"
  }
}
anomaly_info {
  key: "year_week"
  value {
    description: "Examples contain values missing from the schema: 2018_36 (~100%). "
    severity: ERROR
    short_description: "Unexpected string values"
    reason {
      type: ENUM_TYPE_UNEXPECTED_STRING_VALUES
      short_description: "Unexpected string values"
      description: "Examples contain values missing from the schema: 2018_36 (~100%). "
    }
    path {
      step: "year_week"
    }
  }
}
anomaly_name_format: SERIALIZED_PATH



✓ App completed.